<a href="https://colab.research.google.com/github/faizuddin/IBB31103/blob/main/lab_exercise_2_(probability).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probability Exercise

We’re going to calculate the probability a student gets an A (80%+) in math, given they miss 10 or more classes.

## Dataset used in this exercise

*   https://www.kaggle.com/uciml/student-alcohol-consumption (*in this exercise, I have included the one from IBB31103 code repository*)


## Data structure used this exercise

* Numpy (https://numpy.org)  
* Pandas (https://pandas.pydata.org)





In [ ]:
import numpy as np
import pandas as pd

# download file from IBB31103 code repository
!wget https://raw.githubusercontent.com/faizuddin/IBB31103/main/student-mat.csv

# load the file into pandas dataframe
df = pd.read_csv("student-mat.csv")

## Inspect the data using Pandas built-in functions
*   View the first 5 records in the dataset: ``head()``
*   Check the total number of records: ``len()``
*   Basic statistics of the dataset (only for columns with numerical data): ``describe()``

In [ ]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [ ]:
print("Total number of records: \n")
len(df)

Total number of records: 



395

In [ ]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


## Goal
Compute the probability a student gets an A (80%+) in math, given they miss 10 or more classes.

### Extract related columns
From the dataset, we are only concerned with the following columns:
1.   ``absences`` (number of absences)
2.   ``G3`` (final grade from 0 to 20)

Original values in ``G3`` are on a 0-20 scale, therefore we multiply ``G3`` by 5 to get the values on 0-100 scale.




In [ ]:
# Extract columns of interest and put them in absence_grade dataframe
absence_grade = df[["absences", "G3"]]

# View the first 5 records
absence_grade.head()

,absences,G3
0,6,6
1,4,6
2,10,10
3,2,15
4,4,10



### Helper/utility columns
To make things easier we create a couple of booleans columns from ``absence_grade``:
1. ``grade_A`` - 1 if a student achieved 80% or higher as final score or 0 otherwise.
2. ``high_absences`` -  1 if a student missed 10 or more classes or 0 otherwise.
3. ``count`` - to build pivot table

We use function ``where`` from ``numpy`` library to get elements from ``absence_grade`` based on the given conditions (e.g. ``G3`` >= 80, ``absences`` >= 10)

**References**
*   https://numpy.org/doc/stable/reference/generated/numpy.where.html


In [ ]:
# Multiply G3 by 5 and check whether G3 is equal or more than 80, if it does give value 1 to grade_A or 0 otherwise.
absence_grade["grade_A"] = np.where(absence_grade["G3"]*5 >= 80, 1, 0)

# Check whether absences is equal or more than 10, if it does give value 1 to grade_A or 0 otherwise.
absence_grade["high_absences"] = np.where(absence_grade["absences"] >= 10, 1, 0)

# Count
absence_grade["count"] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [ ]:
# Drop absences and G3 columns - we don't need them to compute probability.
absence_grade = absence_grade[["grade_A", "high_absences", "count"]]

# View the first 5 records of absence_grade dataframe
absence_grade.head()

,grade_A,high_absences,count
0,0,0,1
1,0,0,1
2,0,1,1
3,0,0,1
4,0,0,1


## Pivot Table

We use Pandas ``pivot_table`` function to create a spreadsheet-style pivot table as a DataFrame. This allows us to build distribution tables just like how we did in the lecture.

**References**
*   https://en.wikipedia.org/wiki/Pivot_table
*   https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html

In [ ]:
pt = pd.pivot_table(
    absence_grade,
    values="count",
    index=["grade_A"],
    columns=["high_absences"],
    aggfunc=np.size,
    fill_value=0
)

# View the table
pt

high_absences,0,1
grade_A,,
0,277,78
1,35,5


## Calculating Probabilities
In our case:

*   ``P(A)`` is the probability of a grade of 80% or more.
*   ``P(B)`` is the probability of missing 10 or more classes.
*   ``P(A|B)`` is the probability of a 80% grade or more, given missing 10 or more classes.

### Calculation
*   ``P(A)``: those who got 80% or more (35 + 5) /  total students (395)
*   ``P(B)``: those who is absent 10 or more times (78 + 5) / total students (395)
*   ``P(A,B)``: those who got 80% or more **AND** is absent 10 or more times (5) / total students (395)
* ``P(A|B)``: as per conditional probability formula: P(A,B)/P(B)







In [ ]:
# P(A)
A = (pt.iloc[1,0] + pt.iloc[1,1])/(pt.iloc[1,0] + pt.iloc[1,1]+pt.iloc[0,0] + pt.iloc[0,1])

# P(B)
B = (pt.iloc[0,1] + pt.iloc[1,1])/(pt.iloc[1,0] + pt.iloc[1,1]+pt.iloc[0,0] + pt.iloc[0,1])

# P(A,B)
AB = pt.iloc[1,1]/(pt.iloc[1,0] + pt.iloc[1,1]+pt.iloc[0,0] + pt.iloc[0,1])

# P(A|B)
ABcond = AB/B

In [ ]:
print("P(A) = " + str(A))
print("P(B) = " + str(B))
print("P(A,B) = " + str(AB))
print("P(A|B) = " + str(ABcond))

P(A) = 0.10126582278481013
P(B) = 0.21012658227848102
P(A,B) = 0.012658227848101266
P(A|B) = 0.060240963855421686


## Using Bayes' rule

``P(A|B) = P(B|A).P(A) / P(B)``

### Calculation

Additional calculation for ``P(B|A)``, that is proability of missing 10 classes or more, given getting 80% grades or more.

*   ``P(B|A) = P(B,A)/P(A)`` : *probability* of those who got 80% or more **AND** is absent 10 or more times / *probability* of those who got 80% or more.




In [ ]:
# P(B|A). Note that P(A,B) == P(B,A): commutative law
BAcond = AB/A

BayesAB = (BAcond * A) / B

print("Bayes' Rule")
print("P(A|B) = " + str(BayesAB))

Bayes' Rule
P(A|B) = 0.060240963855421686


## Conclusion

Go to class if you want good grades!